In [ ]:
import pandas as pd
%reload_ext watermark
%matplotlib inline

from contextlib import suppress
from datetime import datetime
import os
import yaml

from metapool.metapool import *
from metapool import (make_sample_sheet, find_threshold, autopool)
from metapool.mp_strings import (PM_LIB_WELL_KEY, PM_COMPRESSED_PLATE_NAME_KEY,
    MINIPICO_LIB_CONC_KEY, PM_BLANK_KEY, TELLSEQ_BARCODE_SET_ID_KEY, 
                                 TELLSEQ_BARCODE_ID_KEY)
from metapool.sample_sheet import (
    TELLSEQ_METAG_SHEET_TYPE, TELLSEQ_ABSQUANT_SHEET_TYPE, make_sections_dict)
from metapool.util import get_set_fp, warn_if_fp_exists, EVP_SUFFIX
%watermark -i -v -iv -m -h -p metapool,sample_sheet,openpyxl -u

In [ ]:
! conda list

# Knight Lab TellSeq pipeline notebook

## Part 4 (of 5): Workflow for normalizing DNA

This portion of the notebook will construct an Echo normalization picklist file for the selected barcode set.

As inputs, it requires:
1. A tab-delimited `*_plate_df_A.txt` file containing the quantitations for the entire 384-well plate
2. A yaml file containing the experiment name and info on the included studies

The workflow then:
1. reads in the specified input files
2. calculates the pooling volumes for the samples in the selected barcode set
3. produces an Echo-formatted pick list file for that set
4. produces a tab-delimited `*_plate_df_B_evp_set_*.txt` file containing the plate df for (only) the samples in this barcode set 

### Step 1 of 7: Read in the 384-well plate data and the experiment info

In [ ]:
full_plate_fp = './test_output/QC/Tellseq_plate_df_A.txt'
expt_config_fp = './test_output/QC/Tellseq_expt_info.yml'

In [ ]:
# if the full_plate_fp does not end with "plate_df_A.txt", throw an error
expected_suffix = f"plate_df_A.txt"
if not full_plate_fp.endswith(expected_suffix):
    raise ValueError(f"Expected file ending with '{expected_suffix}'")

In [ ]:
for curr_fp in [full_plate_fp, expt_config_fp]:
    if not os.path.isfile(curr_fp):
        print("Problem! %s is not a path to a valid file" % curr_fp)

In [ ]:
full_plate_df = pd.read_csv(full_plate_fp, sep='\t')
full_plate_df.head()

In [ ]:
with open(expt_config_fp, 'r') as f:
    expt_config = yaml.safe_load(f)

In [ ]:
is_absquant(full_plate_df)

In [ ]:
expt_name = expt_config['experiment_name']
expt_name

In [ ]:
full_studies_info = expt_config['studies']
full_studies_info

### Step 2 of 7: Select the barcode set

In [ ]:
set_ids = full_plate_df[TELLSEQ_BARCODE_SET_ID_KEY].unique()
set_ids

Select the barcode set to process in this notebook and set it below.

In [ ]:
current_set_id = "col19to24"

In [ ]:
plate_df = full_plate_df[full_plate_df[TELLSEQ_BARCODE_SET_ID_KEY] == current_set_id].copy()
plate_df.shape

In [ ]:
plate_df.head()

Verify that there are no duplicate barcodes in the selected plate df.  This must return True.

In [ ]:
plate_df[TELLSEQ_BARCODE_ID_KEY].value_counts().nunique() == 1

In [ ]:
row_col_key = f"{PM_LIB_WELL_KEY}_row"
col_col_key = f"{PM_LIB_WELL_KEY}_col"

In [ ]:
source_well_names = make_compressed_2d_array(
    plate_df, data_col=PM_LIB_WELL_KEY, 
    row_col=row_col_key, col_col=col_col_key)
source_well_names

In [ ]:
unique_projects = plate_df[PM_PROJECT_NAME_KEY].unique()
studies_info = []
for a_study in full_studies_info:
    if a_study[PM_PROJECT_NAME_KEY] in unique_projects:
        studies_info.append(a_study)
studies_info

### Step 3 of 7: Calculate pooling values for MiniPico with autopool

This step will calculate the sample pooling, and update the sample data frame with the pool info.
There are two automated methods to pool:
1. **norm**: This will attempt to generate a normalized pool, automatically inferring the best parameter for pooling.
    - ***pool_failures***:
        - _high_: will pool failures at the highest pooling volume from optimized pooling.
        - _low_: will pool failures at the lowest pooling volume from optimized pooling.

2. **evp**: This will pool an even volume per sample.
    - ***total_vol***: (Optional, Default: 100µL) The total volume to pool, in uL. Each sample will be pooled at 1/N of that volume, where N is total number of samples in the prep.

3. **automate**: (Optional, Default = True) When False, this argument will allow one input parameters for **Legacy** arguments. 

> **Legacy**
> There are legacy parameters to control pooling behaviors when autopool automation (automate=True) returns a poor result. To use these parameters, one must pass automate=False.

>   - **min_conc**: (default: 0) This is the minimum concentration for a sample to be considered for pooling.
    Set to 0 to pool all samples, regardless of concentration. Increasing this will have the 
    effect of increasing pool concentration, at the expense of samples dropping out. 
>   - **floor_conc**: This is the lowest concentration equivalent for which a sample will be 
    accurately pooled. Samples below this concentration will be pooled to the volume that they 
    would have been if they were actually that concentration. For example, if `floor_conc=20`, 
    and a sample at 20 nM pools at 500 nL, a sample at 40 nM will pool at 250 nL but a sample at 
    10 nM will still pool at 500 nL (rather than 1000). Increasing this value will have the effect 
    of increasing pool concentration, but decreasing read counts for low-concentration samples. 
>   - **total_nmol**: This is the total number of molecules to shoot for in the pool. Increasing
    this will increase the overall volume of the pool.


In [ ]:
total_vol = 190

In [ ]:
plate_df = autopool(plate_df,method='evp',total_vol=total_vol)
plate_df.head()

In [ ]:
MINIPICO_POOLED_VOL_KEY = 'MiniPico Pooled Volume'

vols = make_compressed_2d_array(
    plate_df, data_col=MINIPICO_POOLED_VOL_KEY, 
    row_col=row_col_key, col_col=col_col_key
).astype(float)
vols

In [ ]:
#threshold = find_threshold(plate_df[MINIPICO_LIB_CONC_KEY], plate_df[PM_BLANK_KEY])
threshold = find_threshold(plate_df['MiniPico Library Concentration'], plate_df['Blank'])
threshold

In [ ]:
# visualize
print("Floor concentration: {}".format(threshold))
conc, vol = estimate_pool_conc_vol(plate_df[MINIPICO_POOLED_VOL_KEY], plate_df[MINIPICO_LIB_CONC_KEY])
print("Pool concentration: {:.2f}".format(conc))
print("Pool volume: {:.2f}".format(vol))
with suppress(np.linalg.LinAlgError):
    plot_plate_vals(vols)

In [ ]:
sns.scatterplot(x=MINIPICO_LIB_CONC_KEY, y=MINIPICO_POOLED_VOL_KEY,data=plate_df)

### Step 4 of 7: Make equal volume pooling pick list

In [ ]:
evp_picklist_fbase = './test_output/Indices/Tellseq_evp'

In [ ]:
evp_picklist = format_pooling_echo_pick_list(
    vols, max_vol_per_well=30000, source_well_names=source_well_names)

In [ ]:
evp_picklist_fp = get_set_fp(evp_picklist_fbase, current_set_id)
warn_if_fp_exists(evp_picklist_fp)

In [ ]:
with open(evp_picklist_fp,'w') as f:
    f.write(evp_picklist)

!head {evp_picklist_fp}

### Step 5 of 7: Make machine samplesheet for iSeq instrument

In [ ]:
machine_samplesheet_fbase = './test_output/SampleSheets/Tellseq_samplesheet_iseq'

Do not change the below constants unless you really know what you are doing!

In [ ]:
MACHINE_SHEET_TEMPLATE = """[Header],,,,,,,,,,,
Experiment Name,{expt_name},,,,,,,,,,
Investigator Name,Enter the investigator name (optional),,,,,,,,,,
Project Name,{proj_name},,,,,,,,,,
Date,{today_date},,,,,,,,,,
Workflow,GenerateFASTQ,,,,,,,,,,
Library Prep Kit,TELLSEQ,,,,,,,,,,
[Manifest],,,,,,,,,,,
Enter the manifest files used to align to targeted reference regions of the genome. Use the following format.,,,,,,,,,,,
ManifestKey, ManifestFile,,,,,,,,,,
[Reads],,,,,,,,,,,
146,,,,,,,,,,,
146,,,,,,,,,,,
[Settings],,,,,,,,,,,
Enter any analysis settings. See the example setting below,,,,,,,,,,,
Adapter, CTGTCTCTTATACACATCT,,,,,,,,,,
[Data],,,,,,,,,,,
Enter sample information for the run in this section,,,,,,,,,,,
Sample_ID,Sample_Name,Sample_Plate,Description,I7_Index_ID,index,I5_Index_ID,index2,Manifest,GenomeFolder,Sample_Project,Sample_Well
Sample1,,A,A1,7027,NNNNNNNNNNNNNNNNNN,5001,NNNNNNNNNN,,,,"""

In [ ]:
def make_set_machine_sheet_str(a_plate_df, an_expt_name, a_set_id):
    set_expt_name = f"{an_expt_name}_{a_set_id}"
    compressed_plate_name = \
        a_plate_df[PM_COMPRESSED_PLATE_NAME_KEY].unique()[0]
    curr_date = datetime.now().strftime('%Y-%m-%d')
    result = MACHINE_SHEET_TEMPLATE.format(
        expt_name=set_expt_name, proj_name=compressed_plate_name, 
        today_date=curr_date)
    return result

In [ ]:
machine_sheet_str = make_set_machine_sheet_str(
    plate_df, expt_name, current_set_id)

In [ ]:
machine_samplesheet_fp = get_set_fp(machine_samplesheet_fbase, current_set_id)
warn_if_fp_exists(machine_samplesheet_fp)

In [ ]:
with open(machine_samplesheet_fp,'w') as f:
    f.write(machine_sheet_str)
    
!head {machine_samplesheet_fp}

### Step 6 of 7: Make iSeq and NovaSeqX samplesheets for the SPP

In [ ]:
spp_samplesheet_fbase = './test_output/SampleSheets/Tellseq_samplesheet_spp'

In [ ]:
lanes = [1]

In [ ]:
# CONSTANTS: Users, DO NOT CHANGE THESE
# values without consulting with tech team
SHEET_TYPE_VERSIONS = {
    TELLSEQ_METAG_SHEET_TYPE: '10',  # version supporting SampleContext
    TELLSEQ_ABSQUANT_SHEET_TYPE: '10'
}

BIOINFO_BASE = {
    'ForwardAdapter': 'GATCGGAAGAGCACACGTCTGAACTCCAGTCAC',
    'ReverseAdapter': 'GATCGGAAGAGCGTCGTGTAGGGAAAGGAGTGT',
    'library_construction_protocol': 'Knight Lab Kapa HyperPlus',
    # The BarcodesAreRC value is no longer used, but is still checked for
    # by the validation while making the sample sheet, so put in a dummy value
    'BarcodesAreRC': 'True'
}

In [ ]:
# Determine the sample sheet type to make
expt_type = TELLSEQ_ABSQUANT_SHEET_TYPE if is_absquant(plate_df) \
    else TELLSEQ_METAG_SHEET_TYPE
expt_type

In [ ]:
# Extend the metadata dictionary with additional information
metadata_dict_w_sample_context = make_sections_dict(
    plate_df, studies_info, expt_name,
    expt_type, SHEET_TYPE_VERSIONS[expt_type], BIOINFO_BASE)

In [ ]:
iseq_spp_sheet = make_sample_sheet(
    metadata_dict_w_sample_context, plate_df, 'iSeq', lanes)

In [ ]:
iseq_spp_samplesheet_fp = get_set_fp(
    f"{spp_samplesheet_fbase}_iseq", current_set_id)
warn_if_fp_exists(iseq_spp_samplesheet_fp)

In [ ]:
with open(iseq_spp_samplesheet_fp,'w') as f:
    iseq_spp_sheet.write(f)

!head {iseq_spp_samplesheet_fp}

In [ ]:
novaseq_spp_sheet = make_sample_sheet(
    metadata_dict_w_sample_context, plate_df, 'NovaSeqX', lanes)

In [ ]:
novaseq_spp_samplesheet_fp = get_set_fp(
    f"{spp_samplesheet_fbase}_novaseq", current_set_id)
warn_if_fp_exists(novaseq_spp_samplesheet_fp)

In [ ]:
with open(novaseq_spp_samplesheet_fp,'w') as f:
    novaseq_spp_sheet.write(f)

!head {novaseq_spp_samplesheet_fp}

### Step 7 of 7: Write plate set dataframe to file

Save the plate dataframe for this set containing the pooled volume results.

In [ ]:
plate_set_fbase = full_plate_fp.replace("A.txt", f"B")
plate_set_fp = get_set_fp(plate_set_fbase, current_set_id)
plate_set_fp

In [ ]:
warn_if_fp_exists(plate_set_fp)

In [ ]:
plate_df.to_csv(plate_set_fp, sep="\t", index=False)